# Import of Frameworks and Libraries

In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np

le = LabelEncoder()

## Reading the data

In [8]:
st_path = 'C:/Users/lenovo/DataSet/CIC-IDS-2017/GeneratedLabelledFlows/TrafficLabelling/'
st_file = 'Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv'
st_file2 = 'Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv'
# st_file_merged = 'merged.csv'
# st_file = 'Friday-WorkingHours-Morning.pcap_ISCX.csv'
# st_file = 'Monday-WorkingHours.pcap_ISCX.csv'
# st_file = 'Tuesday-WorkingHours.pcap_ISCX.csv'
encoding = 'utf_8'
df_data = pd.read_csv(os.path.join(st_path, st_file), encoding=encoding)
df_data2 = pd.read_csv(os.path.join(st_path, st_file2), encoding=encoding)
df_data.columns

Index(['Flow ID', ' Source IP', ' Source Port', ' Destination IP',
       ' Destination Port', ' Protocol', ' Timestamp', ' Flow Duration',
       ' Total Fwd Packets', ' Total Backward Packets',
       'Total Length of Fwd Packets', ' Total Length of Bwd Packets',
       ' Fwd Packet Length Max', ' Fwd Packet Length Min',
       ' Fwd Packet Length Mean', ' Fwd Packet Length Std',
       'Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s',
       ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max',
       ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std',
       ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean',
       ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags',
       ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags',
       ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s',
       ' Bwd Packets/s', ' Min Packet Length', ' Max Pa

In [3]:
st_path_test_file = 'C:/Users/lenovo/DataSet/MachineLearningCSV/MachineLearningCVE/'
st_file_test_file = 'Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv'
encoding = 'utf_8'
df_data_test_file= pd.read_csv(os.path.join(st_path_test_file, st_file_test_file), encoding=encoding)

### Comparing the data to see if they can be merged

In [26]:
def compare_csv_headers(df_data, df_data_test_file):
    """
    Compare the headers (column names) of two CSV files using pandas and return a boolean indicating if they match.
    """
    data_cols = set(df_data.columns)
    test_cols = set(df_data_test_file.columns)
    
    if data_cols == test_cols:
        return True
    else:
        print("Columns that are in the first CSV file but not in the second: ", data_cols - test_cols)
        print("Columns that are in the second CSV file but not in the first: ", test_cols - data_cols)
        return False

# Example usage
if compare_csv_headers(df_data, df_data_test_file):
    print("The CSV files have the same columns.")
else:
    print("The CSV files have different columns.")

Columns that are in the first CSV file but not in the second:  {' Source IP', 'Flow ID', ' Protocol', ' Timestamp', ' Destination IP', ' Source Port'}
Columns that are in the second CSV file but not in the first:  set()
The CSV files have different columns.


### Merging the data

In [11]:
def merge_csv_files(file1, file2):
    """
    Merge the data from two CSV files using pandas and save the result to a new file in the same directory.
    """
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)
    
    # Merge the DataFrames
    merged_df = pd.concat([df1, df2], axis=0, ignore_index=True)
    
    # Get the directory of the input files
    directory = os.path.dirname(file1)
    
    # Create the output file path
    output_file = os.path.join(directory, 'merged.csv')
    
    # Write the merged DataFrame to a CSV file
    merged_df.to_csv(output_file, index=False)

# Example usage
file1 = 'C:/Users/lenovo/DataSet/CIC-IDS-2017/GeneratedLabelledFlows/TrafficLabelling/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv'
file2 = 'C:/Users/lenovo/DataSet/CIC-IDS-2017/GeneratedLabelledFlows/TrafficLabelling/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv'

merge_csv_files(file1, file2)
print("The data has been merged and saved to merged.csv.")


The data has been merged and saved to merged.csv.


In [4]:
df_data_merged = pd.read_csv(os.path.join(st_path, st_file_merged), encoding=encoding)
df_data_merged.size


43538020

### Identifying the types of attacks within the data

In [4]:
filtered_df = df_data_merged[df_data_merged[' Label'] != 'BENIGN']
label_column = filtered_df[' Label']
unique_labels = label_column.unique()
print(unique_labels)

NameError: name 'df_data_merged' is not defined

In [35]:
df_data.size

45043030

## Feature Selection

In [5]:
# Explain why we are dropping these columns
df_features = df_data.drop(['Flow ID',' Source IP', 'Flow ID', ' Timestamp', ' Destination IP', ' Source Port'], axis=1)  # Features
df_target = df_data[' Label']  # Target variable

df_features_test_file = df_data_test_file  # Features
df_target_test_file = df_data_test_file[' Label']  # Target variable

In [41]:
# Iterate over the columns in the dataframe to check if they are strings
for st_col in df_features.columns:
    if df_features[st_col].dtypes not in ['int64', 'float64']:
        print(df_features[st_col].dtypes)
        df_features[st_col] = le.fit_transform(df_features[st_col])

for st_col in df_features_test_file.columns:
    if df_features_test_file[st_col].dtypes not in ['int64', 'float64']:
        print(df_features_test_file[st_col].dtypes)
        df_features_test_file[st_col] = le.fit_transform(df_features_test_file[st_col])



object
object


In [42]:
# Search for columns with infinite values
lt_columns = df_features[df_features.columns[df_features.max() == np.inf]].columns
lt_columns_test_file = df_features_test_file[df_features_test_file.columns[df_features_test_file.max() == np.inf]].columns


In [44]:
# modify infinite values (10 x max)
for st_column_inf in lt_columns:
    print(st_column_inf)
    df_column_aux = df_features[st_column_inf]
    # identify the max value
    vl_max_aux = df_column_aux[df_column_aux < np.inf].max()
    print(vl_max_aux)
    # .loc is important to modify the value in the dataframe
    df_features.loc[df_features[st_column_inf] == np.inf, st_column_inf] = 10*vl_max_aux
    
for st_column_inf2 in lt_columns_test_file:
    print(st_column_inf2)
    df_column_aux = df_features_test_file[st_column_inf2]
    # identify the max value
    vl_max_aux = df_column_aux[df_column_aux < np.inf].max()
    print(vl_max_aux)
    # .loc is important to modify the value in the dataframe
    df_features_test_file.loc[df_features_test_file[st_column_inf2] == np.inf, st_column_inf2] = 10*vl_max_aux

Flow Bytes/s
20700000000.0
 Flow Packets/s
30000000.0
Flow Bytes/s
2070000000.0
 Flow Packets/s
3000000.0


In [45]:
# check if there are still columns with infinite values

lt_columns = df_features[df_features.columns[df_features.max() == np.inf]].columns
print('columns inf', lt_columns)

lt_columns_test_file = df_features_test_file[df_features_test_file.columns[df_features_test_file.max() == np.inf]].columns
print('columns inf', lt_columns)

columns inf Index([], dtype='object')
columns inf Index([], dtype='object')


In [49]:

# Search for the columns with NaN values
for st_column_nan in df_features.columns:
    df_column_aux = df_features[df_features[st_column_nan].isna()].copy()
    if len(df_column_aux) > 0:
        print(df_column_aux.transpose())
        print(df_target[df_features[st_column_nan].isna()].transpose())
        print(st_column_nan)
        print('The total amount of NaNs are', len(df_features[df_features[st_column_nan].isna()]))
        print(df_features[st_column_nan].describe())
# Drop the rows with NaN values
df_features.dropna(inplace=True)
df_target = df_target[df_target.index.isin(df_features.index)]


for st_column_nan2 in df_features_test_file.columns:
    df_column_aux = df_features_test_file[df_features_test_file[st_column_nan].isna()].copy()
    if len(df_column_aux) > 0:
        print(df_column_aux.transpose())
        print(df_target[df_features_test_file[st_column_nan].isna()].transpose())
        print(st_column_nan)
        print('The total amount of NaNs are', len(df_features_test_file[df_features_test_file[st_column_nan].isna()]))
        print(df_features_test_file[st_column_nan].describe())
# Drop the rows with NaN values
df_features_test_file.dropna(inplace=True)
df_target_test_file = df_target_test_file[df_target_test_file.index.isin(df_features_test_file.index)]

### Scaling the data

In [47]:
# Scale numerical features
scaler = StandardScaler()
mt_features_scaled = scaler.fit_transform(df_features)

mt_features_scaled_test_file = scaler.fit_transform(df_features_test_file)


In [48]:
print(mt_features_scaled)
print(mt_features_scaled_test_file)

[[ 2.32786588 -0.51521605 -0.18640844 ... -0.47836899 -0.39107531
  -1.14464915]
 [ 2.33743334 -0.51521269 -0.25124679 ... -0.47836899 -0.39107531
  -1.14464915]
 [ 2.33748396 -0.5152145  -0.25124679 ... -0.47836899 -0.39107531
  -1.14464915]
 ...
 [ 2.65741191 -0.51521377 -0.25124679 ... -0.47836899 -0.39107531
  -1.14464915]
 [ 2.65477959 -0.51521462 -0.18640844 ... -0.47836899 -0.39107531
  -1.14464915]
 [ 2.65493146 -0.51521399 -0.25124679 ... -0.47836899 -0.39107531
  -1.14464915]]
[[ 2.32786588 -0.51521605 -0.18640844 ... -0.47836899 -0.39107531
  -1.14464915]
 [ 2.33743334 -0.51521269 -0.25124679 ... -0.47836899 -0.39107531
  -1.14464915]
 [ 2.33748396 -0.5152145  -0.25124679 ... -0.47836899 -0.39107531
  -1.14464915]
 ...
 [ 2.65741191 -0.51521377 -0.25124679 ... -0.47836899 -0.39107531
  -1.14464915]
 [ 2.65477959 -0.51521462 -0.18640844 ... -0.47836899 -0.39107531
  -1.14464915]
 [ 2.65493146 -0.51521399 -0.25124679 ... -0.47836899 -0.39107531
  -1.14464915]]


### Splitting the data into training and testing

In [52]:
X_train = mt_features_scaled
y_train = df_target
X_test = mt_features_scaled_test_file
y_test = df_target_test_file

In [37]:
# X_train, X_test, y_train, y_test = train_test_split(mt_features_scaled,
#                                                     df_target,
#                                                     test_size=0.2,
#                                                     random_state=42)

40283     BENIGN
26370       DDoS
50636     BENIGN
52512     BENIGN
219471    BENIGN
           ...  
119882    BENIGN
103697    BENIGN
131935      DDoS
146870    BENIGN
121961      DDoS
Name:  Label, Length: 180592, dtype: object


## Training the model
### Decision Tree Classifier - This is the key part of the code.


In [53]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [20]:
from sklearn.ensemble import RandomForestClassifier

clf2 = RandomForestClassifier()
clf2.fit(X_train, y_train)

RandomForestClassifier()

### Predicting and evaluating the model

### Visualizing the Decision Tree

In [54]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      BENIGN       1.00      1.00      1.00     97714
        DDoS       1.00      1.00      1.00    128027

    accuracy                           1.00    225741
   macro avg       1.00      1.00      1.00    225741
weighted avg       1.00      1.00      1.00    225741



### Visualizing the Random Forest

In [55]:
y_pred2 = clf2.predict(X_test)
print(classification_report(y_test, y_pred2))

ValueError: X has 79 features, but RandomForestClassifier is expecting 76 features as input.

### Confusion Matrix for Random Forest Classifier

In [56]:
# Confusion Matrix
mt_results = np.concatenate((np.matrix(y_pred2), np.matrix(y_test)))
df_results = pd.DataFrame(mt_results, index=['pred', 'test']).transpose()
df_results['equals'] = df_results['test'] == df_results['pred']

vl_equals = len(df_results[df_results['equals'] == True])
vl_len_data = len(df_results)
print('total', vl_equals, vl_len_data, vl_equals/vl_len_data)

df_ddos = df_results[df_results['test'] != 'BENIGN'].copy()
vl_equals = len(df_ddos[df_ddos['equals'] == True])
vl_len_data = len(df_ddos)
print('Attack', vl_equals, vl_len_data, vl_equals/vl_len_data)

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 102443 and the array at index 1 has size 225741

### Confusion Matrix for Decision Tree Classifier

In [57]:
# Confusion Matrix
mt_results = np.concatenate((np.matrix(y_pred), np.matrix(y_test)))
df_results = pd.DataFrame(mt_results, index=['pred', 'test']).transpose()
df_results['equals'] = df_results['test'] == df_results['pred']

vl_equals = len(df_results[df_results['equals'] == True])
vl_len_data = len(df_results)
print('total', vl_equals, vl_len_data, vl_equals/vl_len_data)

df_ddos = df_results[df_results['test'] != 'BENIGN'].copy()
vl_equals = len(df_ddos[df_ddos['equals'] == True])
vl_len_data = len(df_ddos)
print('Attack', vl_equals, vl_len_data, vl_equals/vl_len_data)

total 225741 225741 1.0
Attack 128027 128027 1.0


In [58]:
df_results = pd.DataFrame(y_pred, columns=['pred'])
df_results = pd.concat((df_results, pd.DataFrame(y_test, columns=['test'])), axis=1)
df_results

,pred,test
0,BENIGN,NaN
1,BENIGN,NaN
2,BENIGN,NaN
3,BENIGN,NaN
4,BENIGN,NaN
...,...,...
225736,BENIGN,NaN
225737,BENIGN,NaN
225738,BENIGN,NaN
225739,BENIGN,NaN
